# The PIMPL idiom

## motivation

When changes are made to a header file, all sources including it needs to be recompiled. In large projects and libraries, it can cause build time issues due to the fact that even when a small change to the implementation is made everyone has to wait some time until they compile their code. One way to solve this problem is by using the PImpl Idiom, which hides the implementation in the headers and includes an interface file that compiles instantly.

The PImpl Idiom (Pointer to IMPLementation) is a technique used for separating implementation from the interface. It minimizes header exposure and helps programmers to reduce build dependencies by moving the private data members in a separate class and accessing them through an opaque pointer.

> Remove compilation dependencies on internal class implementations and improve compile times.

# How to implement: 

1. Create a separate class ( or struct ) for implementation
2. Put all private members from the header to that class.
3. Define an Implementation class ( Impl ) in the header file.
4. In the header file create a forward declaration (a pointer), pointing at the implementation class.
5. Define a destructor and a copy/assignment operators.
The reason to declare explicitly a destructor is that when compiling, the smart pointer ( std::unique_ptr ) checks if in the definition of the type exists a visible destructor and throws a compilation error if it’s only forward declared.

Using a smart pointer is a better approach since the pointer takes control over the life cycle of the PImpl.

In [1]:
// foo.h - header file
#include <memory>
class foo
{
  public:
    foo();
    ~foo();
    foo(foo&&);
    foo& operator=(foo&&);
  private:
    class impl;
    std::unique_ptr<impl> pimpl;
};

In [ ]:
// foo.cpp - implementation file
class foo::impl
{
  public:
    void do_internal_work()
    {
      internal_data = 5;
    }
  private:
    int internal_data = 0;
};

foo::foo():pimpl{std::make_unique<impl>()}{
  pimpl->do_internal_work();
}

foo::~foo() = default;
foo::foo(foo&&) = default;
foo& foo::operator=(foo&&) = default;

## Advantages of PImpl:

* Binary Compatibility: The binary interface is independent of the private fields. Making changes to the implementation would not break the dependent code.
* Compilation time: Compilation time drops due to the fact that only the implementation file needs to be rebuilt instead of every client recompiling his file.
* Data Hiding: Can easily hide certain internal details such as implementation techniques and other libraries used to implement the public interface.

## Disadvantages of PImpl:

* Memory Management: Possible increase in memory usage due to more memory allocation than with the default structure which can be critical in embedded software development.
* Maintenance Effort: The maintenance is becoming more complex due to the additional class in order to use pimpl and additional pointer indirection (Interface can be used only via pointer/reference).
* Inheritance: Hidden implementation cannot be inherited, although a class PImpl can.